In [1]:
import pandas as pd
from pathlib import Path
import lxml
from lxml.etree import ElementTree,Element,SubElement
from lxml import etree

In [2]:
def set_xml_values_from_dict(el, a_dict):
    for k, v in a_dict.items():

        subel = el.find(k)
        if subel is not None:
            if v is not None:
                subel.text = str(v)
        else:
            subel = SubElement(el, k)
            subel.text = str(v)

In [3]:
def xml_container_element_to_value_dict(el, convert=True):
    a_dict = dict()
    for sub_el in el.getchildren():
        if len(sub_el.getchildren()) == 0:
            a_dict[sub_el.tag] = sub_el.text
            if convert is True:
                a_dict[sub_el.tag] = convert_xml_text(a_dict[sub_el.tag])
    if 'Index' in a_dict:
        a_dict['Index'] = str(a_dict['Index'])
    return a_dict

def convert_xml_text(text):
    """
    
    Parameters
    ----------
    text : str

    Returns
    -------
    resp : str,int,float
        converted text from xml in native python type.
    """

    if text is None:
        return text
    else:
        if (text.startswith(',') or text.startswith('.')) and text.endswith(tuple([str(el) for el in range(0, 10)])):
            text = '0' + text
        try:

            text_conv = int(text.replace(',', '.'))

        except ValueError:

            try:

                text_conv = float(text.replace(',', '.'))

            except ValueError:

                text_conv = text.replace('\n', '').replace('\t', '')

        return text_conv

In [4]:
path = Path("..")/"modele_donnee"

xml_validator = etree.XMLSchema(file=str((path/"DPEv2.xsd").absolute()))


df_mdd=pd.read_excel(path/'modele_donnee.xlsx')
df_mdd = df_mdd[['description','nom_variable','parent_xpath','type','condition','DPE NEUF']]

enum_table = pd.read_excel(path/'enum_tables.xlsx', sheet_name=None)

valeur_table = pd.read_excel(path/'valeur_tables.xlsx', sheet_name=None)
valeur_table['coef_masque_lointain_non_homogene']=valeur_table['coef_masque_lointain_non_homoge']
del valeur_table['coef_masque_lointain_non_homoge']
enums=[el for el in df_mdd.nom_variable.dropna().values.tolist() if el.startswith('enum')]
enums_without_id =[el for el in enums if not el.endswith('id')]
enums_without_id


enums=[el for el in df_mdd.nom_variable.dropna().values.tolist() if el.startswith('enum')]
enums_without_id =[el for el in enums if not el.endswith('id')]
enums_without_id

enums_dict = {k:v.set_index('id').lib.to_dict() for k,v in enum_table.items() if 'lib' in v and 'id' in v}



mdd_dict = dict()

for grp in df_mdd.parent_xpath.unique():
    mdd_dict[grp]={el:"" for el in df_mdd.loc[df_mdd.parent_xpath==grp].nom_variable.tolist()}


et=ElementTree(Element('dpe'))
dpe = et.getroot()



In [5]:
with open(r'D:\Cstbdata\MONWORKSPACE\Gitlab\py_3cl_dpe\py_3cl_dpe\json_sortie\cas_test_DHUP\cas_APT_1.json','r') as f:
    appt_1=json.load(f)

# dpe

In [6]:
dpe.set("version","4.2.0")

# administratif

In [7]:
administratif=SubElement(dpe,"administratif")

In [8]:
set_xml_values_from_dict(administratif,{
 'date_visite_diagnostiqueur': '2021-07-10',
 'date_etablissement_dpe':  '2021-07-10',
 'nom_proprietaire': 'Hubert Bonisseur de La Bath',
 'nom_proprietaire_installation_commune': "Syndic Saint D'hic",
 'enum_modele_dpe_id': 1,
"enum_version_id":1})


In [9]:
diagnostiqueur=SubElement(administratif,"diagnostiqueur")

In [13]:
set_xml_values_from_dict(diagnostiqueur,{
 'usr_logiciel_id': '3',
 'version_logiciel': '3.2',
 'nom_diagnostiqueur': 'Lamar',
 'prenom_diagnostiqueur': 'Kendrick',
 'mail_diagnostiqueur': 'kendrick.lamar@diagnostiqueur.fr',
 'telephone_diagnostiqueur': '0607080910',
 'adresse_diagnostiqueur': '20 avenue de Ségur 75007 Paris',
 'numero_certification_diagnostiqueur': '????????????',
 'organisme_certificateur': 'Organisme Certificateur'})

In [14]:
geolocalisation=SubElement(administratif,"geolocalisation")

In [15]:
set_xml_values_from_dict(geolocalisation,{'invar_logement': '0151234567', 'idpar': '06163000CE8888',
                                          'immatriculation_copropriete': 'AA1234567'})

In [16]:
addr_expected_fields=mdd_dict['dpe.administratif.geolocalisation.adresses.adresse_bien/adresse_proprietaire/adresse_proprietaire_installation_commune']

In [17]:
from geocoding_tools.addok_utils import get_addok_search

## adresses

In [18]:
adresses=SubElement(geolocalisation,"adresses")

### adresse propriétaire

In [19]:
addr=dict()
addr['adresse_brut']='3 rue roland martin'
addr['code_postal_brut']='94500'
addr['nom_commune_brut']="Champigny Sur Marne"
addr['label_brut']=' '.join([addr['adresse_brut'],addr['code_postal_brut'],addr['nom_commune_brut']])
addr['enum_statut_geocodage_ban_id']=1
addr['ban_date_appel']="2021-07-10"

addr.update({'compl_nom_residence': 'résidence les lilas',
 'compl_ref_batiment': 'A',
 'compl_etage_appartement': 0,
 'compl_ref_cage_escalier': 'A1',
 'compl_ref_logement': 'A12'})

addr_res_dict=get_addok_search(addr['label_brut'])[0]['properties']
addr_res_dict={'ban_'+k:v for k,v in addr_res_dict.items()}

for k in addr_expected_fields:
    if k in addr_res_dict:
        addr[k]=addr_res_dict[k]

In [20]:
adresse_proprietaire=SubElement(adresses,"adresse_proprietaire")

In [21]:
set_xml_values_from_dict(adresse_proprietaire,addr)

In [22]:
addr = dict()

addr['adresse_brut']='116 avenue de Rosny'
addr['code_postal_brut']='93130'
addr['nom_commune_brut']="Noisy Le Sec"
addr['label_brut']=' '.join([addr['adresse_brut'],addr['code_postal_brut'],addr['nom_commune_brut']])
addr['enum_statut_geocodage_ban_id']=1
addr['ban_date_appel']="2021-07-10"

addr.update({'compl_nom_residence': 'résidence des iris',
 'compl_ref_batiment': 'B',
 'compl_etage_appartement':3,
 'compl_ref_logement': '40'})

addr_res_dict=get_addok_search(addr['label_brut'])[0]['properties']
addr_res_dict={'ban_'+k:v for k,v in addr_res_dict.items()}

for k in addr_expected_fields:
    if k in addr_res_dict:
        addr[k]=addr_res_dict[k]

In [23]:
adresse_bien=SubElement(adresses,"adresse_bien")

In [24]:
set_xml_values_from_dict(adresse_bien,addr)

In [25]:
addr = dict()
addr['adresse_brut']='48 Avenue Marceau'
addr['code_postal_brut']='93130'
addr['nom_commune_brut']="Noisy-le-Sec"
addr['label_brut']=' '.join([addr['adresse_brut'],addr['code_postal_brut'],addr['nom_commune_brut']])
addr['enum_statut_geocodage_ban_id']=1
addr['ban_date_appel']="2021-07-10"



addr_res_dict=get_addok_search(addr['label_brut'])[0]['properties']
addr_res_dict={'ban_'+k:v for k,v in addr_res_dict.items()}

for k in addr_expected_fields:
    if k in addr_res_dict:
        addr[k]=addr_res_dict[k]

In [26]:
addr_res_dict

{'ban_label': '48 Avenue Marceau 93130 Noisy-le-Sec',
 'ban_score': 0.9741772727272727,
 'ban_housenumber': '48',
 'ban_id': '93053_6120_00048',
 'ban_name': '48 Avenue Marceau',
 'ban_postcode': '93130',
 'ban_citycode': '93053',
 'ban_x': 660505.01,
 'ban_y': 6865744.45,
 'ban_city': 'Noisy-le-Sec',
 'ban_context': '93, Seine-Saint-Denis, Île-de-France',
 'ban_type': 'housenumber',
 'ban_importance': 0.71595,
 'ban_street': 'Avenue Marceau'}

In [27]:
adresse_proprietaire_installation_commune=SubElement(adresses,"adresse_proprietaire_installation_commune")
set_xml_values_from_dict(adresse_proprietaire_installation_commune,addr)

# identite logement

## carac general

In [28]:
logement=SubElement(dpe,"logement")


In [29]:
caracteristique_generale=SubElement(logement,"caracteristique_generale")


In [30]:

set_xml_values_from_dict(caracteristique_generale,{'annee_construction': 1954,
 'enum_periode_construction_id': 2,
 'enum_methode_application_dpe_log_id': 7,
 'surface_habitable_immeuble': 2500,
 'nombre_niveau_immeuble': 5,
'nombre_appartement': 30,

"hsp":2.8,
                                                   
})

## meteo

In [31]:
meteo=SubElement(logement,"meteo")

set_xml_values_from_dict(meteo,{'enum_zone_climatique_id': 1,
 'enum_classe_altitude_id': 1,
 'batiment_materiaux_anciens': 0})

# enveloppe

In [32]:
enveloppe=SubElement(logement,"enveloppe")


## inertie

In [33]:
inertie=SubElement(enveloppe,"inertie")


In [34]:

set_xml_values_from_dict(inertie,{'inertie_plancher_bas_lourd': 1,
 'inertie_plancher_haut_lourd': 1,
 'inertie_paroi_verticale_lourd': 1,
 'enum_classe_inertie_id': 1})

## murs

In [35]:
mur_collection=SubElement(enveloppe,"mur_collection")


In [36]:
mur=SubElement(mur_collection,"mur")
de=SubElement(mur,"donnee_entree")
di=SubElement(mur,"donnee_intermediaire")


set_xml_values_from_dict(de,{ 'description': 'Mur exterieur sud',
 'tv_coef_reduction_deperdition_id': 1,

 'enum_type_adjacence_id': 1,
 'enum_orientation_id': 1,
 'surface_paroi_totale': 500,
 'surface_paroi_opaque': 300,
 'epaisseur_isolation': 10,
 'resistance_isolation': 3,        
 'epaisseur_structure': 20,
 'enum_type_isolation_id':3 ,
 'enum_materiaux_structure_mur_id': 9,
 'enum_methode_saisie_u0_id':3 ,
 'umur0_saisi':3.5,                            
 'paroi_ancienne': 0,
 'enum_methode_saisie_u_id': 5})

set_xml_values_from_dict(di,{
'b': 1, 'umur': 0.33333, 'umur0': 3.5  })

In [37]:
mur=SubElement(mur_collection,"mur")
de=SubElement(mur,"donnee_entree")
di=SubElement(mur,"donnee_intermediaire")


set_xml_values_from_dict(de,{
 'description': 'mur adjacent',
 'tv_coef_reduction_deperdition_id': 10,
 'enum_type_adjacence_id': 14,
"enum_cfg_isolation_lnc_id":4,
    'surface_aiu': 500.4,
    "surface_aue": 2000,
 'enum_orientation_id': 2,
 'surface_paroi_totale': 500.4,
 'surface_paroi_opaque': 500.4,
 'tv_umur0_id': 45,
     'tv_umur_id': 1,
 'epaisseur_structure': 19,
 'enum_materiaux_structure_mur_id':8,
 'enum_methode_saisie_u0_id': 1,
 'paroi_ancienne':0 ,
 'enum_type_isolation_id': 1,
 'enum_methode_saisie_u_id': 2})

set_xml_values_from_dict(di,{
'b': 0.95, 'umur': 2.5, 'umur0': 2.75})

In [38]:
mur=SubElement(mur_collection,"mur")
de=SubElement(mur,"donnee_entree")
di=SubElement(mur,"donnee_intermediaire")


set_xml_values_from_dict(de,{
 'description': 'mur adjacent lnc',
 'tv_coef_reduction_deperdition_id': 31,
 'surface_aiu':500,
 'surface_aue': 800,
 'enum_cfg_isolation_lnc_id':4,
 'enum_type_adjacence_id': 13,
 'enum_orientation_id': 3,
 'surface_paroi_totale': 500.4,
 'surface_paroi_opaque': 500.4,
 'tv_umur0_id': 45,
 'epaisseur_structure': 19,
 'enum_materiaux_structure_mur_id': 8,
 'enum_methode_saisie_u0_id': 2,
 'paroi_ancienne': 0,
 'enum_type_isolation_id': 2,
 'enum_methode_saisie_u_id': 1})

set_xml_values_from_dict(di,{
'b': 0.8, 'umur': 2.75, 'umur0': 2.75})

## planchers bas

In [39]:
plancher_bas_collection=SubElement(enveloppe,"plancher_bas_collection")


In [40]:
plancher_bas=SubElement(plancher_bas_collection,"plancher_bas")
de=SubElement(plancher_bas,"donnee_entree")
di=SubElement(plancher_bas,"donnee_intermediaire")


set_xml_values_from_dict(de,{'description': 'plancher bas',
 'tv_coef_reduction_deperdition_id': 3,
 'enum_type_adjacence_id': 3,
 'surface_paroi_opaque': 300,
 'tv_upb0_id': 3,
 'enum_type_plancher_bas_id': 3,
 'enum_methode_saisie_u0_id': 1,
 'enum_type_isolation_id': 9,
 'enum_periode_isolation_id': 2,
 'tv_upb_id': 2,
 'enum_methode_saisie_u_id': 2,
 'calcul_ue': 1,
 'perimetre_ue': 200,
 'ue': 0.5})

set_xml_values_from_dict(di,{'b':1, 'upb': 2, 'upb_final': 0.5, 'upb0': 1.45})

## plancher haut

In [41]:
plancher_haut_collection=SubElement(enveloppe,"plancher_haut_collection")
plancher_haut=SubElement(plancher_haut_collection,"plancher_haut")
de=SubElement(plancher_haut,"donnee_entree")
di=SubElement(plancher_haut,"donnee_intermediaire")


In [42]:
set_xml_values_from_dict(de,{'description': 'plancher haut',
 'tv_coef_reduction_deperdition_id':59 ,
 'surface_aiu': 300,
 'surface_aue': 450,
 'enum_cfg_isolation_lnc_id': 4,
 'enum_type_adjacence_id': 11,
 'surface_paroi_opaque': 300,
 'tv_uph0_id': 12,
 'enum_type_plancher_haut_id': 16,
 'enum_methode_saisie_u0_id': 1,
 'enum_type_isolation_id': 4,
 'enum_periode_isolation_id': 7,
 'tv_uph_id': 32,
 'enum_methode_saisie_u_id': 7})
set_xml_values_from_dict(di,{'b':1, 'uph': 0.23, 'uph0': 2.5})

## baie

In [43]:
baie_vitree_collection=SubElement(enveloppe,"baie_vitree_collection")


In [44]:
baie_vitree=SubElement(baie_vitree_collection,"baie_vitree")
de=SubElement(baie_vitree,"donnee_entree")
di=SubElement(baie_vitree,"donnee_intermediaire")


set_xml_values_from_dict(de,{

 'description': 'fenetre battante',
 'tv_coef_reduction_deperdition_id': 1,
 'enum_type_adjacence_id': 1,
 'surface_totale_baie':200 ,
 'nb_baie': 40,
 'tv_ug_id': 1,
 'enum_type_vitrage_id': 1,
 'enum_inclinaison_vitrage_id': 3,
 'enum_methode_saisie_perf_vitrage_id': 1,
 'tv_uw_id':81,
 'enum_type_materiaux_menuiserie_id':6 ,
 'enum_type_baie_id': 5,
 'double_fenetre': 0,
 'enum_type_fermeture_id':1 ,
 'presence_retour_isolation': 0,
 'largeur_dormant': 10,
 'tv_sw_id': 54,
 'enum_type_pose_id': 1,
 'enum_orientation_id': 1,
 'tv_coef_masque_proche_id': 5,
})
mask_l_coll=SubElement(de,"masque_lointain_non_homogene_collection")
mask_l=SubElement(mask_l_coll,"masque_lointain_non_homogene")

set_xml_values_from_dict(mask_l,{'tv_coef_masque_lointain_non_homogene_id':2})
mask_l=SubElement(mask_l_coll,"masque_lointain_non_homogene")

set_xml_values_from_dict(mask_l,{'tv_coef_masque_lointain_non_homogene_id':7})
mask_l=SubElement(mask_l_coll,"masque_lointain_non_homogene")

set_xml_values_from_dict(mask_l,{'tv_coef_masque_lointain_non_homogene_id':5})
mask_l=SubElement(mask_l_coll,"masque_lointain_non_homogene")

set_xml_values_from_dict(mask_l,{'tv_coef_masque_lointain_non_homogene_id':4})

set_xml_values_from_dict(di,{'b': 1,
 'ug': 5.8,
 'uw': 5.8,
 'u_menuiserie': 5.8,
 'sw': 0.64,
 'fe1': 0.5,
 'fe2': 0.08})

In [45]:
baie_vitree=SubElement(baie_vitree_collection,"baie_vitree")
de=SubElement(baie_vitree,"donnee_entree")
di=SubElement(baie_vitree,"donnee_intermediaire")


set_xml_values_from_dict(de,{

 'description': 'porte fenetres',
 'tv_coef_reduction_deperdition_id': 1,
 'enum_type_adjacence_id': 1,
 'surface_totale_baie':60 ,
 'nb_baie': 20,
 'tv_ug_id': 1,
 'enum_type_vitrage_id': 1,
 'enum_inclinaison_vitrage_id': 3,
 'enum_methode_saisie_perf_vitrage_id': 1,
 'tv_uw_id':81,
 'enum_type_materiaux_menuiserie_id':6 ,
 'enum_type_baie_id': 5,
 'double_fenetre': 0,
 'tv_deltar_id': 1,
 'tv_ujn_id':51 ,
 'enum_type_fermeture_id':2 ,
 'presence_retour_isolation': 0,
 'largeur_dormant': 10,
 'tv_sw_id': 54,
 'enum_type_pose_id': 1,
 'enum_orientation_id': 2,
 'tv_coef_masque_proche_id': 1,
})
mask_l_coll=SubElement(de,"masque_lointain_non_homogene_collection")
mask_l=SubElement(mask_l_coll,"masque_lointain_non_homogene")

set_xml_values_from_dict(mask_l,{'tv_coef_masque_lointain_non_homogene_id':2})
mask_l=SubElement(mask_l_coll,"masque_lointain_non_homogene")

set_xml_values_from_dict(mask_l,{'tv_coef_masque_lointain_non_homogene_id':7})
mask_l=SubElement(mask_l_coll,"masque_lointain_non_homogene")

set_xml_values_from_dict(mask_l,{'tv_coef_masque_lointain_non_homogene_id':5})
mask_l=SubElement(mask_l_coll,"masque_lointain_non_homogene")

set_xml_values_from_dict(mask_l,{'tv_coef_masque_lointain_non_homogene_id':4})

set_xml_values_from_dict(di,{'b': 1,
 'ug': 5.8,
 'uw': 5.8,
 'ujn': 4.9,
 'u_menuiserie': 4.9,
 'sw': 0.64,
 'fe1': 0.4,
 'fe2': 0.08,
})

## porte

In [46]:
porte_collection=SubElement(enveloppe,"porte_collection")


In [47]:
porte=SubElement(porte_collection,"porte")
de=SubElement(porte,"donnee_entree")
di=SubElement(porte,"donnee_intermediaire")


set_xml_values_from_dict(de,{
 'description': 'porte',
 'tv_coef_reduction_deperdition_id': 7, 
 'enum_type_adjacence_id': 7,
 'surface_porte': 5,
 'tv_uporte_id': 1,
 'enum_methode_saisie_uporte_id': 1,
 'enum_type_porte_id': 1,
    "enum_cfg_isolation_lnc_id":1,
                         })

set_xml_values_from_dict(di,{'b': 0.95,
'uporte':3.5})

## ets

In [48]:
mdd_dict['dpe.logement.enveloppe.ets_collection.ets.baie_ets_collection.baie_ets.donnee_entree']

{'description': '',
 'reference': '',
 'enum_orientation_id': '',
 'enum_inclinaison_vitrage_id': '',
 'surface_totale_baie': '',
 'nb_baie': ''}

In [49]:
ets_collection=SubElement(enveloppe,"ets_collection")
ets=SubElement(ets_collection,"ets")
de=SubElement(ets,"donnee_entree")
di=SubElement(ets,"donnee_intermediaire")


set_xml_values_from_dict(de,
{'tv_coef_reduction_deperdition_id': 270,
 'description': 'un ets',
 'enum_cfg_isolation_lnc_id': 10,
 'tv_coef_transparence_ets_id': 8})

set_xml_values_from_dict(di,
                        {'coef_transparence_ets': 0.45, 'bver': 0.55})

baie_ets_collection=SubElement(ets,"baie_ets_collection")
baie_ets=SubElement(baie_ets_collection,"baie_ets")
de=SubElement(baie_ets,"donnee_entree")

set_xml_values_from_dict(de,

{
 'enum_orientation_id': 1,
 'enum_inclinaison_vitrage_id': 3,
 'surface_totale_baie': 70,
 'nb_baie': 13})


## PT

In [50]:
pont_thermique_collection=SubElement(enveloppe,"pont_thermique_collection")


# refend
pont_thermique=SubElement(pont_thermique_collection,"pont_thermique")
de=SubElement(pont_thermique,"donnee_entree")
di=SubElement(pont_thermique,"donnee_intermediaire")


set_xml_values_from_dict(de,{'tv_pont_thermique_id': 64,
 'description': 'pont thermique refend/mur',
    'enum_methode_saisie_pont_thermique_id':1,
 'l':20,
 'enum_type_liaison_id': 4})

set_xml_values_from_dict(di,{'k': 0.73})

# pf mur
pont_thermique=SubElement(pont_thermique_collection,"pont_thermique")
de=SubElement(pont_thermique,"donnee_entree")
di=SubElement(pont_thermique,"donnee_intermediaire")


set_xml_values_from_dict(de,{'tv_pont_thermique_id': 72,
 'description': 'pont thermique porte fenetre mur',
'enum_methode_saisie_pont_thermique_id':1,

 'l': 40,
 'enum_type_liaison_id': 5})

set_xml_values_from_dict(di,{'k': 0.29})

# fenetre mur
pont_thermique=SubElement(pont_thermique_collection,"pont_thermique")
de=SubElement(pont_thermique,"donnee_entree")
di=SubElement(pont_thermique,"donnee_intermediaire")


set_xml_values_from_dict(de,{'tv_pont_thermique_id': 76,
 'description': 'pont thermique fenetre mur',
'enum_methode_saisie_pont_thermique_id':1,

 'l': 5,
 'enum_type_liaison_id': 5})

set_xml_values_from_dict(di,{'k': 0.25})

# Pb mur
pont_thermique=SubElement(pont_thermique_collection,"pont_thermique")
de=SubElement(pont_thermique,"donnee_entree")
di=SubElement(pont_thermique,"donnee_intermediaire")


set_xml_values_from_dict(de,{'tv_pont_thermique_id': 1,
 'description': 'pont thermique Plancher bas/mur',
                                 'enum_methode_saisie_pont_thermique_id':1,

 'l': 200,
 'enum_type_liaison_id': 1})

set_xml_values_from_dict(di,{'k': 0.39})


# PH mur
pont_thermique=SubElement(pont_thermique_collection,"pont_thermique")
de=SubElement(pont_thermique,"donnee_entree")
di=SubElement(pont_thermique,"donnee_intermediaire")


set_xml_values_from_dict(de,{'tv_pont_thermique_id': 36,
 'description': 'pont thermique Plancher haut/mur',
                                 'enum_methode_saisie_pont_thermique_id':1,

 'l': 20,
 'enum_type_liaison_id': 3})

set_xml_values_from_dict(di,{'k': 0.3})

# ventilation

In [51]:
ventilation_collection=SubElement(logement,"ventilation_collection")
ventilation=SubElement(ventilation_collection,"ventilation")
de=SubElement(ventilation,"donnee_entree")
di=SubElement(ventilation,"donnee_intermediaire")

In [52]:
set_xml_values_from_dict(de,{'plusieurs_facade_exposee': 0,
"surface_ventile":2500,
 'description': 'ventilation naturelle',
 'tv_q4pa_conv_id': 2,
 'enum_methode_saisie_q4pa_conv_id': 1,
 'tv_debits_ventilation_id': 25,
 'enum_type_ventilation_id': 25,
 'ventilation_post_2012': 0})

set_xml_values_from_dict(di,{
 'q4pa_conv': 2,
 'conso_auxiliaire_ventilation': 0,
 'hperm': 6.31785,
 'hvent': 58.344})

# climatisation

In [53]:
climatisation_collection=SubElement(logement,"climatisation_collection")

climatisation=SubElement(climatisation_collection,"climatisation")
de=SubElement(climatisation,"donnee_entree")
di=SubElement(climatisation,"donnee_intermediaire")

In [54]:
set_xml_values_from_dict(de,{'surface_clim': 2500,
 'ref_produit_fr': 'pac air air individuelle',
 'description': 'blalbl',
 'tv_seer_id': 1,
 'nombre_logement_echantillon': 30,
 'enum_methode_calcul_conso_id': 4,
 'cle_repartition_clim': 1,
 'enum_periode_installation_fr_id': 1,
 'enum_type_energie_id': 1,
'emetteur_plancher_fr':1,
 'enum_type_generateur_fr_id': 1,
 'enum_methode_saisie_carac_sys_id': 1})

set_xml_values_from_dict(di,{'eer': 0.95*3.6, 'besoin_fr': 42, 'conso_fr': 42, 'conso_fr_depensier': 42})

# prod enr


In [55]:
#production_elec_enr=SubElement(logement,"production_elec_enr")


# ECS

##  installation ecs

In [56]:
installation_ecs_collection=SubElement(logement,"installation_ecs_collection")
installation_ecs=SubElement(installation_ecs_collection,"installation_ecs")

de=SubElement(installation_ecs,"donnee_entree")
di=SubElement(installation_ecs,"donnee_intermediaire")

set_xml_values_from_dict(de,{'description': 'installation hybride ballon electrique solaire',
                             "surface_habitable":2500,
                             "nombre_logement":30,
'enum_cfg_installation_ecs_id': 2,
'enum_type_installation_id': 1,
'enum_methode_calcul_conso_id': 4,
'tv_rendement_distribution_ecs_id': 1,
'tv_facteur_couverture_solaire_id': 33,
'enum_methode_saisie_fact_couv_sol_id':1 ,                           
'enum_bouclage_reseau_ecs_id': 1})

set_xml_values_from_dict(di,{'rendement_distribution': 0.93,
 'besoin_ecs': 1627.1638290072408,
 'besoin_ecs_depensier': 1627.1638290072408*1.3,
"fecs":0.26,
"production_ecs_solaire":34.81*80.4*30*0.26,
 'conso_ecs': 34.81*80.4*30,
 'conso_ecs_depensier':34.81*80.4*30*1.3,
})

## generateur ecs

In [57]:
generateur_ecs_collection=SubElement(installation_ecs,"generateur_ecs_collection")
generateur_ecs=SubElement(generateur_ecs_collection,"generateur_ecs")

de=SubElement(generateur_ecs,"donnee_entree")
di=SubElement(generateur_ecs,"donnee_intermediaire")


set_xml_values_from_dict(de,{'enum_type_generateur_ecs_id': 71,
 'ref_produit_generateur_ecs': 'Atlantic ZENEO',
 'description': 'ballon electrique ',
 'enum_usage_generateur_id': 2,
 'enum_type_energie_id': 1,
 'enum_methode_saisie_carac_sys_id': 1,
 'tv_pertes_stockage_id': 4,
 'enum_type_stockage_ecs_id': 3,
 'position_volume_chauffe': 1,
 'volume_stockage': 300})

set_xml_values_from_dict(di,{
 'ratio_besoin_ecs': 1,
 'rendement_generation': 1,
 'rendement_stockage':0.6491,
 'conso_ecs': 34.81*80.4,
 'conso_ecs_depensier':34.81*80.4*1.3,})


# chauffage

## installation chauffage

In [58]:
installation_chauffage_collection=SubElement(logement,"installation_chauffage_collection")
installation_chauffage=SubElement(installation_chauffage_collection,"installation_chauffage")

de=SubElement(installation_chauffage,"donnee_entree")
di=SubElement(installation_chauffage,"donnee_intermediaire")

set_xml_values_from_dict(de,{'surface_chauffee': 2500,
 'description': 'installation de chauffage collective',
 'enum_cfg_installation_ch_id': 1,
 'enum_type_installation_id': 2,
 'enum_methode_calcul_conso_id': 1,
})

set_xml_values_from_dict(di,{'besoin_ch': 8458.035143042967,
 'besoin_ch_depensier':1.3*8458.035143042967,
 'conso_ch': 80.4*119.38543,
 'conso_ch_depensier': 1.3*80.4*119.38543,})

## generateur chauffage

In [59]:
mdd_dict['dpe.logement.installation_chauffage_collection.installation_chauffage.generateur_chauffage_collection.generateur_chauffage.donnee_intermediaire']

{'scop': '',
 'pn': '',
 'qp0': '',
 'pveilleuse': '',
 'temp_fonc_30': '',
 'temp_fonc_100': '',
 'rpn': '',
 'rpint': '',
 'rendement_generation': '',
 'conso_ch': '',
 'conso_ch_depensier': ''}

In [60]:
generateur_chauffage_collection=SubElement(installation_chauffage,"generateur_chauffage_collection")
generateur_chauffage=SubElement(generateur_chauffage_collection,"generateur_chauffage")

de=SubElement(generateur_chauffage,"donnee_entree")
di=SubElement(generateur_chauffage,"donnee_intermediaire")


set_xml_values_from_dict(de,{
'enum_lien_generateur_emetteur_id': 1,
 'enum_type_generateur_ch_id': 85,
 'description': 'chaudière gaz',
 'enum_usage_generateur_id': 1,
 'tv_temp_fonc_100_id': 3,
 'tv_temp_fonc_30_id':21 ,
 'tv_generateur_combustion_id': 1,
     'presence_ventouse': 0,
    "presence_regulation_combustion":1,
 'enum_type_energie_id': 2,
 'position_volume_chauffe': 0,
 'enum_methode_saisie_carac_sys_id': 1})

set_xml_values_from_dict(di,{
 'pn': 240000,
 'qp0': 240000*4/100,
 'pveilleuse': 240,
 'temp_fonc_30': 59,
 'temp_fonc_100':80 ,
 'rpn': 0.88,
 'rpint': 0.84,
 'rendement_generation': 0.8,
 'conso_ch': 119.38543*80.4,
 'conso_ch_depensier':  119.38543*80.4*1.3})


## emetteur chauffage

In [61]:
emetteur_chauffage_collection=SubElement(installation_chauffage,"emetteur_chauffage_collection")
emetteur_chauffage=SubElement(emetteur_chauffage_collection,"emetteur_chauffage")

de=SubElement(emetteur_chauffage,"donnee_entree")
di=SubElement(emetteur_chauffage,"donnee_intermediaire")


set_xml_values_from_dict(de,{
    "surface_chauffee":2500,
    'enum_lien_generateur_emetteur_id': 1,
    'tv_intermittence_id': 179,
 'description': 'convecteur electrique ancien',
 'tv_rendement_emission_id': 7,
 'tv_rendement_distribution_ch_id':3 ,
 'tv_rendement_regulation_id': 12,
 'enum_type_emission_distribution_id': 28,
 'enum_equipement_intermittence_id':6,
 'enum_type_regulation_id': 2,
 'enum_type_chauffage_id': 2,
 'enum_temp_distribution_ch_id':4,
})

set_xml_values_from_dict(di,{
 'rendement_emission': 0.95,
 'rendement_distribution': 0.85,
 'rendement_regulation': 0.95,
 'i0': 0.93,
})


# sortie


In [62]:
sortie=SubElement(logement,"sortie")


## deperdition

In [63]:
deperdition=SubElement(sortie,"deperdition")
deper_dict={ 'hperm': 6.31785,
 'hvent': 58.344,
 'deperdition_renouvellement_air': 64.66,
 'deperdition_mur': 8.959999999999999+36.0+18.2,
 'deperdition_plancher_bas': 0,
 'deperdition_plancher_haut': 0,
 'deperdition_baie_vitree': 17.099999999999998+ 17.099999999999998+4.7,
'deperdition_porte':2.44,

 'deperdition_pont_thermique': 1.022+0.35+6.51+0.43+8.6+1.37+3.01,
}
deper_dict['deperdition_enveloppe']=0
for el in ['deperdition_mur', 'deperdition_plancher_bas', 'deperdition_plancher_haut', 'deperdition_baie_vitree', 'deperdition_pont_thermique']:
    deper_dict["deperdition_enveloppe"]+=deper_dict[el]
set_xml_values_from_dict(deperdition,deper_dict)


## apports

In [64]:
apport_et_besoin=SubElement(sortie,"apport_et_besoin")
set_xml_values_from_dict(apport_et_besoin,{'surface_sud_equivalente': 0.127,
 'apport_solaire_fr': 0,
 'apport_interne_fr': 0,
 'apport_solaire_ch':43009/1000 ,
 'apport_interne_ch': 1701360/1000,
 'fraction_apport_gratuit_ch': 0.165,
 'fraction_apport_gratuit_depensier_ch': 0.19999,
 'pertes_distribution_ecs_recup': 999,
 'pertes_distribution_ecs_recup_depensier':999,
 'pertes_stockage_ecs_recup': 42,
 'pertes_generateur_ch_recup': 0,
 'pertes_generateur_ch_recup_depensier': 0,
 'nadeq': 2,
 'v40_ecs_journalier': 115,
 'v40_ecs_journalier_depensier': 163,
'besoin_ch': 8458.035143042967,
 'besoin_ch_depensier':1.3*8458.035143042967,
 'besoin_ecs': 1627.1638290072408,
 'besoin_ecs_depensier': 1627.1638290072408*1.3,
 'besoin_fr': 0,
 'besoin_fr_depensier': 0})



## conso

In [65]:
ef_conso=SubElement(sortie,"ef_conso")

set_xml_values_from_dict(ef_conso,{
 'conso_ch': 80.4*119.38543,
 'conso_ch_depensier': 1.3*80.4*119.38543,
 'conso_ecs': 34.81*80.4,
 'conso_ecs_depensier':34.81*80.4*1.3,
 'conso_eclairage': 42,
 'conso_auxiliaire_generation_ch': 42,
 'conso_auxiliaire_generation_ch_depensier': 42,
 'conso_auxiliaire_distribution_ch': 42,
 'conso_auxiliaire_generation_ecs': 42,
 'conso_auxiliaire_generation_ecs_depensier': 42,
 'conso_auxiliaire_distribution_ecs': 42,
 'conso_auxiliaire_distribution_fr': 42,
 'conso_auxiliaire_ventilation': 0,
 'conso_totale_auxiliaire': 42,
 'conso_fr': 0,
 'conso_fr_depensier': 0,
 'conso_5_usages': 42,
 'conso_5_usages_m2': 397,

}
                        )

In [66]:
ep_conso=SubElement(sortie,"ep_conso")

set_xml_values_from_dict(ep_conso,{
 'ep_conso_ch': 80.4*119.38543,
 'ep_conso_ch_depensier': 1.3*80.4*119.38543,
 'ep_conso_ecs': 34.81*80.4,
 'ep_conso_ecs_depensier':34.81*80.4*1.3,
 'ep_conso_eclairage': 42,
 'ep_conso_auxiliaire_generation_ch': 42,
 'ep_conso_auxiliaire_generation_ch_depensier': 42,
 'ep_conso_auxiliaire_distribution_ch': 42,
 'ep_conso_auxiliaire_generation_ecs': 42,
 'ep_conso_auxiliaire_generation_ecs_depensier': 42,
 'ep_conso_auxiliaire_distribution_ecs': 42,
 'ep_conso_auxiliaire_distribution_fr': 42,
 'ep_conso_auxiliaire_ventilation': 0,
 'ep_conso_totale_auxiliaire': 42,
 'ep_conso_fr': 0,
 'ep_conso_fr_depensier': 0,
 'ep_conso_5_usages': 42,
 'ep_conso_5_usages_m2': 42,
     'classe_conso_energie': 'F'
})

In [67]:
emission_ges=SubElement(sortie,"emission_ges")

set_xml_values_from_dict(emission_ges,{
 'emission_ges_ch': 80.4*119.38543,
 'emission_ges_ch_depensier': 1.3*80.4*119.38543,
 'emission_ges_ecs': 34.81*80.4,
 'emission_ges_ecs_depensier':34.81*80.4*1.3,
 'emission_ges_eclairage': 42,
 'emission_ges_auxiliaire_generation_ch': 42,
 'emission_ges_auxiliaire_generation_ch_depensier': 42,
 'emission_ges_auxiliaire_distribution_ch': 42,
 'emission_ges_auxiliaire_generation_ecs': 42,
 'emission_ges_auxiliaire_generation_ecs_depensier': 42,
 'emission_ges_auxiliaire_distribution_ecs': 42,
 'emission_ges_auxiliaire_distribution_fr': 42,
 'emission_ges_auxiliaire_ventilation': 0,
 'emission_ges_totale_auxiliaire': 42,
 'emission_ges_fr': 0,
 'emission_ges_fr_depensier': 0,
 'emission_ges_5_usages': 42,
 'emission_ges_5_usages_m2': 22.88,
'classe_emission_ges': 'D'
})

In [68]:
cout=SubElement(sortie,"cout")

set_xml_values_from_dict(cout,{
 'cout_ch': 80.4*119.38543,
 'cout_ch_depensier': 1.3*80.4*119.38543,
 'cout_ecs': 34.81*80.4,
 'cout_ecs_depensier':34.81*80.4*1.3,
 'cout_eclairage': 42,
 'cout_auxiliaire_generation_ch': 42,
 'cout_auxiliaire_generation_ch_depensier': 42,
 'cout_auxiliaire_distribution_ch': 42,
 'cout_auxiliaire_generation_ecs': 42,
 'cout_auxiliaire_generation_ecs_depensier': 42,
 'cout_auxiliaire_distribution_ecs': 42,
 'cout_auxiliaire_distribution_fr': 42,
 'cout_auxiliaire_ventilation': 0,
 'cout_total_auxiliaire': 42,
 'cout_fr': 0,
 'cout_fr_depensier': 0,
 'cout_5_usages': 42,
})

In [69]:
production_electricite=SubElement(sortie,"production_electricite")
set_xml_values_from_dict(production_electricite,{
 'production_pv':  0,
 'conso_elec_ac':0,
     'conso_elec_ac_ch':0,
 'conso_elec_ac_ecs':0,
 'conso_elec_ac_fr':0,
 'conso_elec_ac_eclairage':0,
 'conso_elec_ac_auxiliaire':0,
 'conso_elec_ac_autre_usage':0,

})

In [70]:
sortie_par_energie_collection=SubElement(sortie,"sortie_par_energie_collection")
sortie_par_energie=SubElement(sortie_par_energie_collection,"sortie_par_energie")

set_xml_values_from_dict(sortie_par_energie,{'conso_ch':0,
 'conso_ecs':  1.3*80.4*119.38543,
 'conso_5_usages':42,
 'enum_type_energie_id': 1,
 'emission_ges_ch': 42,
 'emission_ges_ecs': 42,
 'emission_ges_5_usages': 42,
 'cout_ch': 42,
 'cout_ecs': 42,
 'cout_5_usages': 42})

In [71]:
sortie_par_energie=SubElement(sortie_par_energie_collection,"sortie_par_energie")

set_xml_values_from_dict(sortie_par_energie,{'conso_ch':80.4*119.38543,
 'conso_ecs':  0,
 'conso_5_usages':42,
 'enum_type_energie_id': 2,
 'emission_ges_ch': 42,
 'emission_ges_ecs': 42,
 'emission_ges_5_usages': 42,
 'cout_ch': 42,
 'cout_ecs': 42,
 'cout_5_usages': 42})

In [72]:
confort_ete=SubElement(sortie,"confort_ete")

set_xml_values_from_dict(confort_ete,{'isolation_toiture': 0,
 'protection_solaire_exterieure': 1,
 'aspect_traversant': 0,
 'brasseur_air': 0,
 'enum_indicateur_confort_ete_id': 2})

In [73]:
qualite_isolation=SubElement(sortie,"qualite_isolation")

set_xml_values_from_dict(qualite_isolation,{'ubat': 42,
 'qualite_isol_enveloppe': 4,
 'qualite_isol_mur': 4,
 'qualite_isol_plancher_haut_toit_terrasse': 4,
 'qualite_isol_plancher_bas': 4,
 'qualite_isol_menuiserie': 4})

# descriptifs et fiches techniques

In [74]:
descriptif_enr_collection=SubElement(dpe,"descriptif_enr_collection")


In [75]:
descriptif_simplifie_collection=SubElement(dpe,"descriptif_simplifie_collection")


In [76]:
descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,
{'description': "mur sud en briques pleines doubles avec lame d'air non isolé; mur est en briques pleines simples donnant sur une circulation",
 'enum_categorie_descriptif_simplifie_id':1})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'plancher intermédiaire non déperditif donnant sur un autre logement adjacent',
                                                        'enum_categorie_descriptif_simplifie_id': 2})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'plancher intermédiaire non déperditif donnant sur un autre logement adjacent',
                                                        'enum_categorie_descriptif_simplifie_id': 3})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,
                         {'description': 'porte en bois pleine simple; Fenêtre battante bois simple vitrage orienté sud;Porte Fenêtre bois simple vitrage orienté sud',
                          'enum_categorie_descriptif_simplifie_id': 4})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'convecteurs electriques anciens(système individuel)',
                                                        'enum_categorie_descriptif_simplifie_id': 5})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': "ballon éléctrique vertical 300L", 
                                                        'enum_categorie_descriptif_simplifie_id': 6})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'Sans Object', 'enum_categorie_descriptif_simplifie_id': 7})

descriptif_simplifie=SubElement(descriptif_simplifie_collection,"descriptif_simplifie")
set_xml_values_from_dict(descriptif_simplifie,{'description': 'ventilation naturelle par conduit', 'enum_categorie_descriptif_simplifie_id': 8})


In [77]:
fiche_technique_collection=SubElement(dpe,"fiche_technique_collection")

fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 1})


sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")


for description,enum_origine_donnee_id,valeur in [
  ('surface',2,'80m²'),
  ("type d'adjacence",2,'extérieur'),
('matériau mur',2,'briques pleines doubles'),
    ('état isolation',3,'non isolé'),
]:
    sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
    set_xml_values_from_dict(sous_fiche_technique,
    {'description': description,
     'enum_origine_donnee_id': enum_origine_donnee_id,
    'valeur':valeur
    })


fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 1})
sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")

for description,enum_origine_donnee_id,valeur in [
  ('surface',2,'51m²'),
  ("type d'adjacence",2,'extérieur'),
('matériau mur',2,'bloc de béton plein'),
    ('état isolation',3,'isolé (ITI)'),
]:

    sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
    set_xml_values_from_dict(sous_fiche_technique,
    {'description': description,
     'enum_origine_donnee_id': enum_origine_donnee_id,
    'valeur':valeur
    })



# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 1})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "mur est en briques pleines simple non isolé donnant sur un immeuble a usage autre qu'habitation U = 3.5 W/m².K, b=0.2 S = 15m²",
#  'enum_origine_donnee_id': 2})



fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 4})

sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
"Fenêtre battante orientée sud simple vitrage Ug = 5.8 W/m².K,Uw = 4.7 W/m².K, b=1"
for description,enum_origine_donnee_id,valeur in [
  ('surface',2,'5m²'),
  ("type d'adjacence",2,'extérieur'),
('type vitrage',2,'Double vitrage'),
]:
    sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
    set_xml_values_from_dict(sous_fiche_technique,
    {'description': description,
     'enum_origine_donnee_id': enum_origine_donnee_id,
    'valeur':valeur
    })


# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 4})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Porte Fenêtre battante orientée sud simple vitrage équipé d'une jalousie accordéon Ug = 5.8 W/m².K,Uw = 4.7,Ujn = 3.8 W/m².K, b=1",
#            "detail_origine_donnee":"mesure du type de vitrage au vitromètre",                                    
#  'enum_origine_donnee_id': 2})


# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 4})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Porte Fenêtre battante orientée sud simple vitrage équipé d'une jalousie accordéon Ug = 5.8 W/m².K,Uw = 4.7,Ujn = 3.8 W/m².K, b=1",
#            "detail_origine_donnee":"mesure du type de vitrage au vitromètre",                                    
#  'enum_origine_donnee_id': 2})


# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 5})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Porte Simple en bois U = 3.5 W/m²/K",
# 'enum_origine_donnee_id': 2})



# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Pont thermique refend/mur l= 2.8 k=0.365 W/m/K",
# 'enum_origine_donnee_id': 2})


# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Pont thermique refend/mur l= 2.8 k=0.365 W/m/K",
# 'enum_origine_donnee_id': 2})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})


# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Pont thermique fenetre/mur l= 5 k=0.25 W/m/K",
# 'enum_origine_donnee_id': 2})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': "Pont thermique pf/mur l= 21 k=0.365 W/m/K",
# 'enum_origine_donnee_id': 2})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 6})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': 'pont thermique Plancher intermediaire/mur l= 16 k=0.43 W/m/K',
# 'enum_origine_donnee_id': 2})



# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 7})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': 'convecteurs electriques anciens Rg=1 Re=0.95 Rr=0.9 Rd=1',
# 'enum_origine_donnee_id': 2})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 8})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': 'ballon electrique vertical atlantic zeneo 300L Rg=1 Rs=0.64 Rd=0.93',
# 'enum_origine_donnee_id': 3})

# fiche_technique=SubElement(fiche_technique_collection,"fiche_technique")
# set_xml_values_from_dict(fiche_technique,{'enum_categorie_fiche_technique_id': 10})

# sous_fiche_technique_collection=SubElement(fiche_technique,"sous_fiche_technique_collection")
# sous_fiche_technique=SubElement(sous_fiche_technique_collection,"sous_fiche_technique")
# set_xml_values_from_dict(sous_fiche_technique,{'description': 'Ventilation naturelle par conduit ',
# 'enum_origine_donnee_id': 2})

# justificatifs

In [78]:
justificatif_collection=SubElement(dpe,"justificatif_collection")
justificatif=SubElement(justificatif_collection,"justificatif")
set_xml_values_from_dict(justificatif,
{'description': "notice technique du ballon d'eau chaude atlantic zeneo", 'enum_type_justificatif_id':10 })

# gestes entretien

In [79]:
descriptif_geste_entretien_collection=SubElement(dpe,"descriptif_geste_entretien_collection")
descriptif_geste_entretien=SubElement(descriptif_geste_entretien_collection,"descriptif_geste_entretien")
set_xml_values_from_dict(descriptif_geste_entretien,
{'enum_picto_geste_entretien_id': 9,
 'description': "detarter et vidanger le ballon d'eau chaude tous les 5 ans",
 'categorie_geste_entretien': 'entretien du chauffe-eau'})

# pack travaux

In [80]:
descriptif_travaux = SubElement(dpe,'descriptif_travaux')
commentaire_travaux = SubElement(descriptif_travaux,'commentaire_travaux')
commentaire_travaux.text = """Des travaux peuvent vous permettre d’améliorer significativement l’efficacité énergétique de votre
logement et ainsi de faire des économies d’énergie, d’améliorer son confort, de le valoriser et de le
rendre plus écologique. Le pack de travaux vous permet de réaliser les travaux prioritaires,
et le pack d’aller vers un logement très performant."""

pack_travaux_collection=SubElement(descriptif_travaux,"pack_travaux_collection")
pack_travaux=SubElement(pack_travaux_collection,"pack_travaux")
set_xml_values_from_dict(pack_travaux,
{'enum_num_pack_travaux_id': 1,
 'conso_5_usages_apres_travaux': 99,
 'cout_pack_travaux_min': 999,
 'cout_pack_travaux_max': 9999})

travaux_collection=SubElement(pack_travaux,"travaux_collection")
travaux = SubElement(travaux_collection,'travaux')
set_xml_values_from_dict(travaux,
{'enum_lot_travaux_id': 1,
 'description_travaux': 'isolation murs ITE 15 cm',
 'avertissement_travaux': 'permis de construire',
 'performance_recommande': 'R = 3,75 m2. K/W'})

travaux = SubElement(travaux_collection,'travaux')
set_xml_values_from_dict(travaux,
{'enum_lot_travaux_id': 4,
 'description_travaux': 'remplacement des vitrages par du double vitrage performant',
 'performance_recommande': 'U = 2.4 W/m²/K'})
                         
pack_travaux=SubElement(pack_travaux_collection,"pack_travaux")
set_xml_values_from_dict(pack_travaux,
{'enum_num_pack_travaux_id': 2,
 'conso_5_usages_apres_travaux': 99,
 'cout_pack_travaux_min': 999,
 'cout_pack_travaux_max': 9999})

travaux_collection=SubElement(pack_travaux,"travaux_collection")
travaux = SubElement(travaux_collection,'travaux')
set_xml_values_from_dict(travaux,
{'enum_lot_travaux_id':5,
 'description_travaux': 'remplacement du système de chauffage par une pompe à chaleur',
 'avertissement_travaux': 'permis de construire',
 'performance_recommande': 'SCOP=4.2'})

                         
                         
pack_travaux=SubElement(pack_travaux_collection,"pack_travaux")
set_xml_values_from_dict(pack_travaux,
{'enum_num_pack_travaux_id': 3,
 'conso_5_usages_apres_travaux': 99,
 'cout_pack_travaux_min': 999,
 'cout_pack_travaux_max': 9999})

travaux_collection=SubElement(pack_travaux,"travaux_collection")
travaux = SubElement(travaux_collection,'travaux')
set_xml_values_from_dict(travaux,
{'enum_lot_travaux_id': 1,
 'description_travaux': 'isolation murs ITE 15 cm',
 'avertissement_travaux': 'permis de construire',
 'performance_recommande': 'R = 3,75 m2. K/W'})
                         
travaux = SubElement(travaux_collection,'travaux')
set_xml_values_from_dict(travaux,
{'enum_lot_travaux_id': 4,
 'description_travaux': 'remplacement des vitrages par du double vitrage performant',
 'performance_recommande': 'U = 2.4 W/m²/K'})
                         
                         
travaux = SubElement(travaux_collection,'travaux')
set_xml_values_from_dict(travaux,
{'enum_lot_travaux_id':5,
 'description_travaux': 'remplacement du système de chauffage par une pompe à chaleur',
 'avertissement_travaux': 'permis de construire',
 'performance_recommande': 'SCOP=4.2'})


# valid

In [81]:
et.write(str((path/'exemples_metier'/'cas_test_immeuble_1_valid.xml')), pretty_print=True, xml_declaration=True, encoding='utf-8')

In [84]:
import lxml

In [85]:



is_valid = xml_validator.validate(et)

In [86]:
et

In [87]:
str(xml_validator.error_log).split('\n')

["<string>:0:0:ERROR:SCHEMASV:SCHEMAV_ELEMENT_CONTENT: Element 'diagnostiqueur': Missing child element(s). Expected is one of ( version_moteur_calcul, entreprise_diagnostiqueur ).",
 "<string>:0:0:ERROR:SCHEMASV:SCHEMAV_ELEMENT_CONTENT: Element 'geolocalisation': Missing child element(s). Expected is ( adresses ).",
 "<string>:0:0:ERROR:SCHEMASV:SCHEMAV_ELEMENT_CONTENT: Element 'geolocalisation': This element is not expected.",
 "<string>:0:0:ERROR:SCHEMASV:SCHEMAV_ELEMENT_CONTENT: Element 'donnee_entree': Missing child element(s). Expected is one of ( reference, surface_aiu, surface_aue, enum_cfg_isolation_lnc_id, tv_umur0_id, umur_saisi, enum_type_doublage_id, enum_periode_isolation_id, tv_umur_id ).",
 "<string>:0:0:ERROR:SCHEMASV:SCHEMAV_ELEMENT_CONTENT: Element 'donnee_entree': Missing child element(s). Expected is one of ( reference, umur0_saisi, umur_saisi, enum_type_doublage_id, enum_periode_isolation_id, resistance_isolation, epaisseur_isolation ).",
 "<string>:0:0:ERROR:SCHEM

In [88]:
is_valid

False

In [399]:
import requests

dpe.find('*enum_version_id').text='2'

r=requests.post('http://localhost:5000/controle_coherence',data=etree.tostring(dpe))

In [400]:
len(r.json()['warning_saisie'])

3

In [401]:
for error in r.json()['erreur_logiciel']:
    print(error['message'])
    print(error['objets_concerne'])

In [402]:
for warning in r.json()['warning_saisie']:
    print(warning['message'])
    print(warning['objets_concerne'])


pour le calcul du b d'une paroi de type mur celle ci est considérée isolé : Oui
alors que l'isolation de la paroi de type mur est {2: 'Non isolé'}

['/dpe/logement/enveloppe/mur_collection/mur[1]/donnee_entree/enum_type_isolation_id']

pour le calcul du b d'une paroi de type mur celle ci est considérée isolé : Oui
alors que l'isolation de la paroi de type mur est {2: 'Non isolé'}

['/dpe/logement/enveloppe/mur_collection/mur[2]/donnee_entree/enum_type_isolation_id']

une baie vitrée orientée {2: 'Nord'}
ne correspond à aucun mur donnant sur l'exterieur ou sur véranda de même orientation. 
Ceci n'est possible que dans certains cas atypiques :
- cas de paroi totalement vitrée 
- cas de velux sur un toit incliné de même orientation qu'un mur sur local non chauffé ou adjacent donnant sur un local d'habitation
- cas d'une paroi vitrée donnant sur un local non chauffé. 
- cas d'un vitrage sur un débord d'une facade. 

['/dpe/logement/enveloppe/baie_vitree_collection/baie_vitree[2]/donnee_en

In [403]:
for warning in r.json()['warning_logiciel']:
    print(warning['message'])
    print(warning['objets_concerne'])


des énergies consommées par des générateurs ne sont pas dans les données de sorties par énergie
energies manquantes : 
{14: 'autre combustible fossile'} 

['/dpe/logement/sortie/sortie_par_energie_collection/sortie_par_energie[1]/enum_type_energie_id', '/dpe/logement/sortie/sortie_par_energie_collection/sortie_par_energie[2]/enum_type_energie_id', '/dpe/logement/installation_ecs_collection/installation_ecs/generateur_ecs_collection/generateur_ecs/donnee_entree/enum_type_energie_id', '/dpe/logement/installation_chauffage_collection/installation_chauffage/generateur_chauffage_collection/generateur_chauffage/donnee_entree/enum_type_energie_id', '/dpe/logement/climatisation_collection/climatisation/donnee_entree/enum_type_energie_id']
